### 中醫醫案症狀詞抽取
- model: CRF
- 主症與兼症合併
- 詞位標注: OSBIE
- 移除671個詞沒有完全對應的資料點
- 以句點為分隔拆句子
- postag使用nltk

In [ ]:
import json
import nltk
import numpy as np
from sklearn.metrics import classification_report

In [ ]:
def getTagListWithString(str):
    list1 = []
    str = str.replace('，',' ')
    str = str.replace('、',' ')
    str = str.replace(',',' ')
    str_list = str.split(" ")
    
    for s in str_list:
        s = s.strip()
        if len(s) > 0:
            list1.append(s)
    
    #list2 = list(filter(None, second_list))

    return list1

In [ ]:
def getLabelWithWordIndex(i,ne_index_list):
    for (ne_idx,ne_len) in ne_index_list: 
        if i == ne_idx:
            return "B-S"
        elif i > ne_idx and i < (ne_idx + ne_len - 1):
            return "I-S"
        elif i == (ne_idx + ne_len - 1):
            return "E-S"
            
    return "O"

In [ ]:
import pynlpir  # 引入依赖包
pynlpir.open()  # 打开分词器


filename = "case_summary.csv"
with open(filename) as f:
    lines = f.readlines()

docs = []
c = 0
t = 0
for idx,line in enumerate(lines):
    cols = line.strip().split(",")
    case_content = cols[2].replace(" ","")
    sentences = case_content.split("。")
    
    summary_str = cols[3].strip()
    
    if idx == 0 or len(summary_str) == 0 :
        continue
        
    #處理每個doc的summary(主症和兼症)
    summary_str = summary_str.strip('"')
    summary_str = summary_str.replace('""','"')
    summary_str = summary_str.replace('#',',')
    #print(summary_str)
        
    summary_json = json.loads(summary_str)
        
    keys = summary_json.keys()
    if "main" in keys:
        main = summary_json["main"]
        main_list = getTagListWithString(main)
        #print(main_list)
    if "second" in keys:
        second = summary_json["second"]
        second_list = getTagListWithString(second)
        #print(second_list)
        
    if len(second_list) > 0:
        main_list.extend(second_list)
    
    #原文編碼有問題，跳過
    if idx in [64,223,251,389,480,673,744,751,769,1422,1541,1853]:
            continue
    
    #症狀詞作斷詞
    ne_list = []
    for term in main_list:
        term_seg_list = pynlpir.segment(term)
        ne_list.append(term_seg_list)
    
    #每個醫案拆句
    for sent in sentences:
        sent_seg_list = pynlpir.segment(sent)
        for i,w in enumerate(sent_seg_list):
            labs_list = getLabe(sent_seg_list,i,ne_list)
        docs.append(labs)

    
print(len(docs))

In [ ]:
def getLab():
    for ne in ne_list:
                if
            texts.append((w,label))

In [ ]:
### 計算平均句長
total_wc = 0
for doc in docs:
    total_wc += len(doc)
    
ave = total_wc / 48901
print(ave)

In [ ]:
data = docs

In [ ]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]
    
    if postag is None:
            postag = 'none'

    # Common features for all words
    features = [
        'bias',
        'word=' + word,
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]
        
    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        
        if postag1 is None:
            postag1 = 'none'
            
        features.extend([
            '-1:word=' + word1,
            '-1.word.isdigit=%s' % word.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        
        if postag1 is None:
            postag1 = 'none'
            
        features.extend([
            '+1:word=' + word1,
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [ ]:
from sklearn.model_selection import train_test_split

# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
print(len(X_train))
print(len(X_test))

In [ ]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

In [ ]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')

y_pred_train = [tagger.tag(xseq) for xseq in X_train]
y_pred_test = [tagger.tag(xseq) for xseq in X_test]

In [ ]:
# Create a mapping of labels to indices
labels = {"O": 0,"S": 1}

# training data F1-score
predictions_train = np.array([labels[tag] for row in y_pred_train for tag in row])
truths_train = np.array([labels[tag] for row in y_train for tag in row])
print(classification_report(
    truths_train, predictions_train,
    target_names=["O", "S"]))

# test data F1-score
predictions_test = np.array([labels[tag] for row in y_pred_test for tag in row])
truths_test = np.array([labels[tag] for row in y_test for tag in row])
print(classification_report(
    truths_test, predictions_test,
    target_names=["O", "S"]))